In [1]:
import numpy as np
import pandas as pd
import scipy.io

In [2]:
raw_data = pd.read_csv("data/raw_data.csv")
private_signals = scipy.io.loadmat('data/game_design.mat')['chosen_signals']

In [3]:
columns = ['sid', 'trial', 'network', 'stage', 'sum private', 'who', 'color', 'degree', 'RD', 'action']
dfs = []
for sid in raw_data['SID'].unique():
    print(sid)
    for trial in raw_data.query('SID==@sid')['game_order'].unique():
        trialdata = raw_data.query('SID==@sid & game_order==@trial')
        network = trialdata['game_index'].unique()[0]
        private = 2*private_signals[network-1] - np.ones((7))
        sum_private = np.sum(private)
        for stage in range(4):
            if stage==0:
                color = trialdata.query("stage==0 & event_type==2")['color'].to_numpy()[0]
                degree = trialdata.query("stage==1 & event_type==5")['degree'].to_numpy()[0]
                RD = 0
                who = trialdata.query("stage==1 & event_type==5")['observed_node'].to_numpy()[0]
                act = trialdata.query("stage==0 & event_type==4")['color'].to_numpy()[0]
                df = pd.DataFrame([[sid, trial, network, stage, sum_private, who, 2*color-1, degree, RD, 2*act-1]], columns=columns)
                dfs.append(df)
            else:
                stagedata = trialdata.query("stage==@stage")
                whos = stagedata.query("event_type==6")['observed_node'].to_numpy()
                for who in whos:
                    whodata = stagedata.query("observed_node==@who & event_type==6")
                    color = whodata['color'].to_numpy()[0]
                    degree = whodata['degree'].to_numpy()[0]
                    RD = whodata["neighbors' RD"].to_numpy()[0]
                    act = stagedata.query("event_type==4")['color'].unique()[0]
                    df = pd.DataFrame([[sid, trial, network, stage, sum_private, who, 2*color-1, degree, RD, 2*act-1]], columns=columns)
                    dfs.append(df)
        
data = pd.concat(dfs, ignore_index=True)
data.to_pickle("data/human2.pkl")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
18
19
20
21
23
24
25
26
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
101
102
103
104
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
134
135
136
137
138
140
141
142
143
144
145
146
147
148
149
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [2]:
data = pd.read_pickle("data/human2.pkl")

In [3]:
data

,sid,trial,network,stage,sum private,who,color,degree,RD,action
0,1,1,9,0,-1.0,2,1,3,0.00000,1
1,1,1,9,1,-1.0,6,-1,3,0.23077,1
2,1,1,9,1,-1.0,5,-1,3,0.23077,1
3,1,1,9,1,-1.0,7,1,4,0.30769,1
4,1,1,9,2,-1.0,5,-1,3,0.23077,-1
...,...,...,...,...,...,...,...,...,...,...
86780,224,40,9,1,-1.0,6,-1,3,0.33333,-1
86781,224,40,9,2,-1.0,6,-1,3,0.33333,1
86782,224,40,9,2,-1.0,7,1,4,0.44444,1
86783,224,40,9,3,-1.0,7,1,4,0.44444,1
